In [4]:
# 相关包与其路径初配置
import sys
from pathlib import Path

# 获取当前文件的路径
current_path = Path().absolute()
# 添加项目根目录（包含 docx_parser 和 llm_structuring 的目录）到系统路径
root_path = current_path.parent.parent.parent
sys.path.append(str(root_path))

from doc_analysis.docx_parser.pipeline import DocxParserPipeline
from doc_analysis.docx_parser._03_element_extractor import ElementType
from doc_analysis.doc_structurer._03_tree_builder import TreeBuilder
from doc_analysis.doc_structurer.doc_tree_retriever import DocTreeRetriever



In [5]:
# 文档路径
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case3_北京铁运投标人须知.docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case3st_北京铁运投标人须知.docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case 9 _样式测试.docx"
doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case8：招标文件-第1包：一级压榨花生油.docx"
#doc_path = "C:/Users/huiwa/Documents/_CursorChat_Extra/Langchain_Jupyter/.LoRA微调/招标文件_微调库/WORD/[2].docx"
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/Case6_中国区零食包框架供应商采购项目招采文件.docx"

#### <font color="red">1. 获取Document Elements</font>

In [6]:
# 获取Document Elements
pipeline = DocxParserPipeline(doc_path)
elements = pipeline.process()

2025-02-13 14:41:32,718 - doc_analysis.docx_parser._01_xml_loader - INFO - 从 C:\Users\huiwa\Downloads\文本分析测试\CaseTest\case8：招标文件-第1包：一级压榨花生油.docx 成功提取 raw XML content
2025-02-13 14:41:32,719 - doc_analysis.docx_parser.pipeline - INFO - 成功加载文档: C:\Users\huiwa\Downloads\文本分析测试\CaseTest\case8：招标文件-第1包：一级压榨花生油.docx
2025-02-13 14:41:32,736 - doc_analysis.docx_parser._02_xml_parser - INFO - Successfully parsed XML content
2025-02-13 14:41:32,737 - doc_analysis.docx_parser._02_xml_parser - INFO - DocxXMLParser initialized successfully
2025-02-13 14:41:32,737 - doc_analysis.docx_parser.pipeline - INFO - 成功解析XML结构
2025-02-13 14:41:32,747 - doc_analysis.docx_parser._03_element_extractor - INFO - Starting element extraction...
2025-02-13 14:41:32,747 - doc_analysis.docx_parser._03_element_extractor - DEBUG - Found 928 total elements
2025-02-13 14:41:32,747 - doc_analysis.docx_parser._03_element_extractor - DEBUG - Processing element 1, type: ElementType.PARAGRAPH
2025-02-13 14:41:32,748 - doc_ana

In [7]:
from pprint import pprint
print(type(elements))
#pprint(elements)

<class 'list'>


In [8]:
pprint(elements[0])

ParagraphElement(element_type=<ElementType.PARAGRAPH: 'paragraph'>,
                 sequence_number=1,
                 content='北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目',
                 raw_xml='<w:p '
                         'xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" '
                         'xmlns:w14="http://schemas.microsoft.com/office/word/2010/wordml" '
                         'xmlns:wpc="http://schemas.microsoft.com/office/word/2010/wordprocessingCanvas" '
                         'xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" '
                         'xmlns:o="urn:schemas-microsoft-com:office:office" '
                         'xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships" '
                         'xmlns:m="http://schemas.openxmlformats.org/officeDocument/2006/math" '
                         'xmlns:v="urn:schemas-microsoft-com:vml" '
                         'xmlns:wp14="http://schemas.mic

document_extractors.py里的转化

In [10]:
# 将提取的元素转换为可序列化的字典列表
serializable_elements = [
    {
        'element_type': str(element.element_type),
        'sequence_number': int(element.sequence_number),
        'content': element.content,
        'indentation': getattr(element, 'indentation', None),
        'is_heading': getattr(element, 'is_heading', False),
        'heading_level': getattr(element, 'heading_level', None),
        'heading_type': getattr(element, 'heading_type', None),
        'heading_info': getattr(element, 'heading_info', None),
        'alignment': getattr(element, 'alignment', None),
        'indentation':getattr(element,'indentation',None),
        'is_toc': getattr(element, 'is_toc', False),
        'toc_info': getattr(element,'toc_info',None),
        'first_line_tabs': getattr(element, 'first_line_tabs', None),
        'has_nested': getattr(element, 'has_merged', None),
        'has_merged': getattr(element, 'has_merged', None)
    }
    for element in elements
]

#### <font color="red">2. 构建文档树</font>

In [11]:
# 构建文档树
tree_builder = TreeBuilder(serializable_elements)
doc_structure = tree_builder.build_to_level(target_level=2)

In [12]:
nodes = doc_structure.doc_nodes
print(f"{doc_structure.doc_tree.node_id}, root length: {doc_structure.doc_tree.branch_length}")
for node in nodes:
    print(f"{node.node_id}, {node.node_type}, level: {node.level}, " 
          f"node_length: {node.node_length}, branch_length: {node.branch_length}, "
          f"#nodes: {node.ttl_nodes_in_branch}, content: {node.element.content[:20]}")


-1, root length: 36112
1, content_node, level: None, node_length: 30, branch_length: None, #nodes: None, content: 北京京铁运恒采购供应站有限公司 2024
2, content_node, level: None, node_length: 4, branch_length: None, #nodes: None, content: 招标文件
3, content_node, level: None, node_length: 54, branch_length: None, #nodes: None, content:         项目名称：北京京铁运恒采
4, content_node, level: None, node_length: 12, branch_length: None, #nodes: None, content: 项目编号：DLXM-2024-148-0
5, content_node, level: None, node_length: 39, branch_length: None, #nodes: None, content: 招 标 人：北京京铁运恒采购供应站有限公
6, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content:                 2024
7, content_node, level: None, node_length: 6, branch_length: None, #nodes: None, content:                     
8, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content: 第一章  招标公告
9, content_node, level: None, node_length: 10, branch_length: None, #nodes: None, content: 第二章  招标需求
10, conte

In [44]:
# 添加文档树检索器
doc_tree_retriever = DocTreeRetriever(doc_structure.doc_tree)

In [45]:
# 获取文档所有内容
doc_tree_retriever.get_all_content()

['北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目',
 '招标文件',
 '        项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油）',
 '项目编号：DLXM-2024-148-01',
 '招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司',
 '                2024 年 3 月',
 '                    目    录',
 '第一章  招标公告',
 '第二章  招标需求',
 '第三章  投标人须知',
 '前附表',
 '一、总则',
 '二、招标文件',
 '三、投标文件的编制',
 '四、开标',
 '五、评标',
 '六、定标',
 '七、合同授予',
 '八、履约保证金',
 '九、招标代理服务费的收取',
 '第四章  评标办法及评分标准',
 '附件一：资格审查表',
 '附件二：符合性审查表',
 '附件三：评分标准',
 '第五章  合同条款及格式',
 '第六章  投标文件格式',
 '附件 1 投标函',
 '附件 2 投标保证书',
 '附件 3 无违法记录声明',
 '附件 4 法定代表人授权书',
 '附件 5 投标人情况简介',
 '附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表',
 '附件 7 投标一览表',
 '附件 8 分项报价表',
 '附件 9 投标方案',
 '附件 10 投标人资格声明',
 '附件 11 资格证明文件',
 '附件 12 与投标人存在“关系”的其他法人单位信息',
 '附件 13 投标人认为必要的其他证明文件',
 '附件 14 技术规格偏离表',
 '附件 15 商务条款偏离表',
 '附件 16 拟投入本项目的人员配备表',
 '附件 17 中标服务费承诺书',
 '附件 18 关于投标保证金的声明',
 '附件 19 开标会声明书',
 '第一章 招标公告',
 '根据《中华人民共和国招标投标法》、《中华人民共和国招标投标法实施条 例》等规定， 北京中外建工程管理有限公司受招标人委托，就北京京铁运恒采购 供应站有限公司 2024 年端午节物资采购项目（第一包：一级压榨花生油）进行  国内公开招标采购，欢迎合

In [46]:
# 打印所有标题节点
doc_tree_retriever.print_list_of_all_heading_nodes_by_tree()

root （51 子nodes，其中 6 子标题nodes）
  ├─ 46 第一章 招标公告  （48 子nodes, 其中 0 子标题nodes）
  ├─ 95 第二章 招标需求  （28 子nodes, 其中 0 子标题nodes）
  ├─ 124 第三章 投标人须知  （10 子nodes, 其中 10 子标题nodes）
  |    |  ├─ 125 前附表  （4 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 130 一、总则  （28 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 159 二、招标文件  （15 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 175 三、投标文件的编制  （80 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 256 四、开标  （10 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 267 五、评标  （26 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 294 六、定标  （3 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 298 七、合同授予  （1 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 300 八、履约保证金  （1 子nodes, 其中 0 子标题nodes）
  |    |  └─ 302 九、招标代理服务费的收取  （5 子nodes, 其中 0 子标题nodes）
  ├─ 308 第四章 评标办法及评分标准  （46 子nodes, 其中 3 子标题nodes）
  |    |  ├─ 352 附件一：资格审查表  （2 子nodes, 其中 0 子标题nodes）
  |    |  ├─ 355 附件二：符合性审查表  （1 子nodes, 其中 0 子标题nodes）
  |    |  └─ 357 附件三：评分标准  （8 子nodes, 其中 0 子标题nodes）
  ├─ 366 第五章 合同条款及格式  （134 子nodes, 其中 0 子标题nodes）
  └─ 501 第六章 投标文件格式  （32 子nodes, 其中 19 子标题nodes）
     

In [47]:
# 获取指定层级的内容
doc_tree_retriever.get_headings_by_level(level=1)

['第一章 招标公告',
 '第二章 招标需求',
 '第三章 投标人须知',
 '第四章 评标办法及评分标准',
 '第五章 合同条款及格式',
 '第六章 投标文件格式']

总结：构建的文档树 有doc_structure封装， 里面有doc_structure.doc_nodes 和 doc_structure.doc_tree 两种可访问数据形式。


#### <font color="red">3. **文档树 分析**</font>


In [48]:
# 所有标题节点
title_nodes = [node for node in doc_structure.doc_nodes if node.element.is_heading]
for node in title_nodes:

    indent = "    " * node.level
    print(node.node_id, node.path_title_id, indent, node.element.content,node.ttl_nodes_in_branch, node.branch_length,)



46 1      第一章 招标公告 48 2306
95 2      第二章 招标需求 28 1620
124 3      第三章 投标人须知 183 10008
125 3.1          前附表 4 2731
130 3.2          一、总则 28 1193
159 3.3          二、招标文件 15 629
175 3.4          三、投标文件的编制 80 3219
256 3.5          四、开标 10 375
267 3.6          五、评标 26 1187
294 3.7          六、定标 3 163
298 3.8          七、合同授予 1 73
300 3.9          八、履约保证金 1 45
302 3.10          九、招标代理服务费的收取 5 327
308 4      第四章 评标办法及评分标准 57 7023
352 4.1          附件一：资格审查表 2 1022
355 4.2          附件二：符合性审查表 1 488
357 4.3          附件三：评分标准 8 3397
366 5      第五章 合同条款及格式 134 7682
501 6      第六章 投标文件格式 153 6832
515 6.1          附件 1 投标函 16 485
532 6.2          附件 2 投标保证书 7 217
540 6.3          附件 3 无违法记录声明 7 82
548 6.4          附件 4 法定代表人授权书 7 231
556 6.5          附件 5 投标人情况简介 2 1299
559 6.6          附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表 2 154
562 6.7          附件 7 投标一览表 7 413
570 6.8          附件 8 分项报价表 8 369
579 6.9          附件 9 投标方案 9 187
589 6.10          附件 10 投标人资格声明 6 206
596 6.11          附件 11 资格证明文件 1

In [49]:
# 文档树最大层级
max_level = max((node.level for node in doc_structure.doc_nodes if node.level is not None), default=None)
print(max_level)

2


##### <font color=orange> 3.1 评估哪些节点还可能是标题？ 正则式反向排除 </font>

In [50]:
# 从文本特征初步评估文本为标题的可能性， 使用了HeadNodePreScreener的pre_screen_nodes()方法
doc_nodes = doc_structure.doc_nodes
head_node_pre_screener = HeadNodePreScreener(doc_nodes)
head_node_pre_screener.pre_screen_nodes()


In [51]:
# Print 展示 所有Nodes的 pre_screening 的结果
print("="*50)
content_count = sum(1 for node in doc_nodes if node.enrich_pre_screened_result == "content")
undetermined_count = sum(1 for node in doc_nodes if node.enrich_pre_screened_result == "undetermined")
print(f"以下为{len(doc_nodes)} 个样本的测试结果, 确定非标题个数: {content_count}, 可能是标题个数: {undetermined_count}")
print("="*50)
for node in doc_nodes:
    print(node.node_id, node.enrich_pre_screened_result, node.element.content)

以下为654 个样本的测试结果, 确定非标题个数: 0, 可能是标题个数: 0
1 [待定]   北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目
2 [待定]   招标文件
3 [正文]         项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油）
4 [待定]   项目编号：DLXM-2024-148-01
5 [待定]   招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司
6 [待定]                   2024 年 3 月
7 [目录]                     目    录
8 [目录] 第一章  招标公告
9 [目录] 第二章  招标需求
10 [目录] 第三章  投标人须知
11 [目录] 前附表
12 [目录] 一、总则
13 [目录] 二、招标文件
14 [目录] 三、投标文件的编制
15 [目录] 四、开标
16 [目录] 五、评标
17 [目录] 六、定标
18 [目录] 七、合同授予
19 [目录] 八、履约保证金
20 [目录] 九、招标代理服务费的收取
21 [目录] 第四章  评标办法及评分标准
22 [目录] 附件一：资格审查表
23 [目录] 附件二：符合性审查表
24 [目录] 附件三：评分标准
25 [目录] 第五章  合同条款及格式
26 [目录] 第六章  投标文件格式
27 [目录] 附件 1 投标函
28 [目录] 附件 2 投标保证书
29 [目录] 附件 3 无违法记录声明
30 [目录] 附件 4 法定代表人授权书
31 [目录] 附件 5 投标人情况简介
32 [目录] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
33 [目录] 附件 7 投标一览表
34 [目录] 附件 8 分项报价表
35 [目录] 附件 9 投标方案
36 [目录] 附件 10 投标人资格声明
37 [目录] 附件 11 资格证明文件
38 [目录] 附件 12 与投标人存在“关系”的其他法人单位信息
39 [目录] 附件 13 投标人认为必要的其他证明文件
40 [目录] 附件 14 技术规格偏离表
41 [目录] 附件 15 商务条款偏离表
42 [目录] 附件 16

##### <font color=orange>挑选末端标题节点 </font>

In [52]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
branch_end_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and not any(child.node_type == "title_node" for child in node.children)  
                    ]

for i, node in enumerate(branch_end_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)

0 46 [章] 第一章 招标公告
1 95 [章] 第二章 招标需求
2 125 [附件] 前附表
3 130 [节] 一、总则
4 159 [节] 二、招标文件
5 175 [节] 三、投标文件的编制
6 256 [节] 四、开标
7 267 [节] 五、评标
8 294 [节] 六、定标
9 298 [节] 七、合同授予
10 300 [节] 八、履约保证金
11 302 [节] 九、招标代理服务费的收取
12 352 [附件] 附件一：资格审查表
13 355 [附件] 附件二：符合性审查表
14 357 [附件] 附件三：评分标准
15 366 [章] 第五章 合同条款及格式
16 515 [附件] 附件 1 投标函
17 532 [附件] 附件 2 投标保证书
18 540 [附件] 附件 3 无违法记录声明
19 548 [附件] 附件 4 法定代表人授权书
20 556 [附件] 附件 5 投标人情况简介
21 559 [附件] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
22 562 [附件] 附件 7 投标一览表
23 570 [附件] 附件 8 分项报价表
24 579 [附件] 附件 9 投标方案
25 589 [附件] 附件 10 投标人资格声明
26 596 [附件] 附件 11 资格证明文件
27 607 [附件] 附件 12 与投标人存在“关系”的其他法人单位信息
28 611 [附件] 附件 13 投标人认为必要的其他证明文件
29 612 [附件] 附件 14 技术规格偏离表
30 618 [附件] 附件 15 商务条款偏离表
31 624 [附件] 附件 16 拟投入本项目的人员配备表
32 630 [附件] 附件 17 中标服务费承诺书
33 636 [附件] 附件 18 关于投标保证金的声明
34 646 [附件] 附件 19 开标会声明书


In [53]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
branch_end_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and not any(child.enrich_pre_screened_result == "[章]" 
                                or child.enrich_pre_screened_result == "[节]" 
                                or child.enrich_pre_screened_result == "[小节]" for child in node.children)  
                    ]

for i, node in enumerate(branch_end_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)



0 46 [章] 第一章 招标公告
1 95 [章] 第二章 招标需求
2 125 [附件] 前附表
3 130 [节] 一、总则
4 159 [节] 二、招标文件
5 175 [节] 三、投标文件的编制
6 256 [节] 四、开标
7 267 [节] 五、评标
8 294 [节] 六、定标
9 298 [节] 七、合同授予
10 300 [节] 八、履约保证金
11 302 [节] 九、招标代理服务费的收取
12 308 [章] 第四章 评标办法及评分标准
13 352 [附件] 附件一：资格审查表
14 355 [附件] 附件二：符合性审查表
15 357 [附件] 附件三：评分标准
16 366 [章] 第五章 合同条款及格式
17 501 [章] 第六章 投标文件格式
18 515 [附件] 附件 1 投标函
19 532 [附件] 附件 2 投标保证书
20 540 [附件] 附件 3 无违法记录声明
21 548 [附件] 附件 4 法定代表人授权书
22 556 [附件] 附件 5 投标人情况简介
23 559 [附件] 附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表
24 562 [附件] 附件 7 投标一览表
25 570 [附件] 附件 8 分项报价表
26 579 [附件] 附件 9 投标方案
27 589 [附件] 附件 10 投标人资格声明
28 596 [附件] 附件 11 资格证明文件
29 607 [附件] 附件 12 与投标人存在“关系”的其他法人单位信息
30 611 [附件] 附件 13 投标人认为必要的其他证明文件
31 612 [附件] 附件 14 技术规格偏离表
32 618 [附件] 附件 15 商务条款偏离表
33 624 [附件] 附件 16 拟投入本项目的人员配备表
34 630 [附件] 附件 17 中标服务费承诺书
35 636 [附件] 附件 18 关于投标保证金的声明
36 646 [附件] 附件 19 开标会声明书


In [54]:
# 添加筛选条件：如节点长度>2000tokens，节点子元素>3, 节点内容为非附件； 待优化，目前先不考虑
selected_branch_end_nodes = [node for node in branch_end_nodes 
                   if node.branch_length > 1000
                   and node.ttl_nodes_in_branch > 3
                   and node.enrich_pre_screened_result != "[附件]"]
for node in selected_branch_end_nodes:
    print(node.node_id, node.enrich_pre_screened_result, node.element.content)

46 [章] 第一章 招标公告
95 [章] 第二章 招标需求
130 [节] 一、总则
175 [节] 三、投标文件的编制
267 [节] 五、评标
308 [章] 第四章 评标办法及评分标准
366 [章] 第五章 合同条款及格式
501 [章] 第六章 投标文件格式


In [55]:
#找到末端标题节点（有的chapter, 有的是section, 有的是小节）
mid_branch_nodes = [node for node in doc_structure.doc_nodes 
                    if node.element.is_heading 
                    and node.children is not None
                    and any(child.node_type == "content_node" for child in node.children) and any(child.node_type == "title_node" for child in node.children)  
                    ]

for i, node in enumerate(mid_branch_nodes):
    print(i, node.node_id, node.enrich_pre_screened_result, node.element.content)

0 308 [章] 第四章 评标办法及评分标准
1 501 [章] 第六章 投标文件格式


##### <font color = orange> 按特定格式 组织 章节文本  （考虑单独函数）</font>

In [56]:
# 计算文本的token数量
import tiktoken
def count_tokens(text: str) -> int:
    """计算文本的token数量"""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))

In [57]:
# 为节点生成Parts用于大模型分析，每个Part都是特定格式的文本
parts = []

for node in branch_end_nodes:
    # 构建文本片段
    part = {}
    text_part = []
    # 添加标题部分
    heading_type = "章" if node.element.heading_level == 1 else "节" if node.element.heading_level == 2 else "小节"
    text_part.append(f"章节编号：{node.node_id}")
    text_part.append(f"章节标题：{node.element.content}")
    
    # 添加正文部分
    text_part.append("正文内容：")
    
    # 遍历子节点，添加内容
    for child in node.children:
        if child.enrich_pre_screened_result == "[附件]":
            text_part.append(f"附件部分：{child.element.content} （这里只展示附件名称）")
        elif child.element.element_type == ElementType.TABLE:
            text_part.append(f"表格：{child.element.content}")
        else:
            text_part.append(f"段落：{child.element.content}")
    # 将所有部分用换行符连接成一个字符串
    final_text = "\n".join(text_part)
    part['content'] = final_text
    part['type'] = node.enrich_pre_screened_result
    part['part_length'] = count_tokens(final_text)

    parts.append(part)



In [58]:
# 打印第一个片段查看效果
print(parts[18]['type'])
print(parts[18]['part_length'])
print(parts[18]['content'])

[附件]
578
章节编号：515
章节标题：附件 1 投标函
正文内容：
段落：致：招标人
段落：根据贵方为                 项目招标采购货物及服务的招标要求，签字代 表          （全名、职务）经正式授权并代表投标人                  （投 标人名称、地址）提交下述文件正本 一 份和副本       份及电子版      份。
段落：1、开标一览表
段落：2、分项报价表
段落：3、投标方案
段落：4、投标须知要求提供的全部文件
段落：5、投标保证金，金额为人民币       元。 据此函，签字代表宣布同意如下：
段落：6、附投标报价表中规定的应提供和交付的货物投标总价为人民币       元。
段落：7、投标人将按招标文件的规定履行合同责任和义务。
段落：8、投标人已详细审查全部招标文件， 包括修改文件（如需要修改）以及全部 参考资料和有关附件。我们完全理解并同意放弃对这方面有不明及误解的权利。
段落：9、投标有效期为                    日历天。
段落：10、在规定的开标日期后，投标人在投标有效期内撤回投标，其投标保证金 将被贵方没收。
段落：11、投标人同意提供按照贵方可能要求的与其投标有关的一切数据或资料， 完全理解贵方不一定接受最低价格的投标或收到的任何投标。
段落：12、本投标有关的一切正式往来通讯请寄：
段落：地址：                                      邮编： 电话：                                      传真： 投标人名称（盖章）：
段落：法定代表人或被授权代表(签字或盖章):                      日期：


<font color=orange> 大模型分析，输出章节 摘要、章节内容、作用、关键行动。 </font>




In [59]:
# prompt 片段， 用于每个章节的分析
Appendix = """
请为我分析以下附件的文本内容，提供它的简明扼要的信息，以json格式输出如下：
[{
 "章节编号": "章节编号",
 "章节标题": "章节标题",
 "附件类型": "模板|补充说明",
 "摘要": "一句话概括附件核心内容",
 "内容包括": "关键信息点列举", 
 "解读环节用途": "招标文件解读环节的作用",
 "编制环节用途": "投标文件编制环节的作用",
 "关键行动": [
   {
     "行动名称": "...",
     "时间": "...",
     "地点": "...",
     "内容": "..."
   }
 ]
}]

文本内容：
<text>
"""

general_prompt = """
请为我分析以下章节的文本内容，提供它的简明扼要的信息，以json格式输出如下：
[{
 "章节编号": "章节编号",
 "章节标题": "章节标题",
 "摘要": "一句话概括附件核心内容",
 "内容包括": "关键信息点列举", 
 "解读环节用途": "招标文件解读环节的作用",
 "编制环节用途": "投标文件编制环节的作用",
 "关键行动": [  // 注:仅包含编制投标文件之外的关键行动
   {
     "行动名称": "...",
     "时间": "...",
     "地点": "...",
     "内容": "..."
   }
 ]
}]

文本内容：
<text>
"""

prompts = {
    "appendix": Appendix,
    "chapter": general_prompt,
    "section": general_prompt,
    "sub_section": general_prompt,
}

In [60]:
# 直接赋值大模型分析的结果
results = [{'关键行动': [{'内容': '发售招标文件，售价500元人民币，需通过电汇或网上银行转账购买。',
            '地点': '北京中外建工程管理有限公司',
            '时间': '2024年3月22日至2024年3月26日，上午9:00-11:00；下午1:30-4:30',
            '行动名称': '招标文件发售'},
           {'内容': '投标人需提交20万元人民币的投标保证金。',
            '地点': '北京中外建工程管理有限公司',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标保证金提交'},
           {'内容': '投标人需在此时间前邮寄或现场递交投标文件。',
            '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座15层',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标文件递交'},
           {'内容': '采用线上方式组织开标。',
            '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座14层',
            '时间': '2024年4月11日09时00分',
            '行动名称': '开标'}],
  '内容包括': '项目编号、采购组织类型、采购方式、招标最高限价、采购内容及技术要求、投标人资格要求、招标公告发布平台、招标文件发售细节、投标保证金要求、投标截止时间和地点、开标时间和地点、评标办法、业务咨询联系方式。',
  '摘要': '北京中外建工程管理有限公司受委托公开招标采购一级压榨花生油，明确了项目编号、采购方式、预算、技术要求、投标人资格、公告发布、招标文件发售、投标保证金、投标截止、开标时间和地点以及评标办法。',
  '章节标题': '第一章 招标公告',
  '章节编号': '46',
  '编制环节用途': '指导投标人准备和提交符合要求的投标文件，包括技术规格、商务条款和法律要求。',
  '解读环节用途': '帮助潜在投标人理解招标要求和流程，确保投标文件符合规定。'},
 {'关键行动': [{'内容': '投标人需提供3桶一级压榨花生油样品，样品应与招标文件要求一致。',
            '地点': '指定地点',
            '时间': '投标文件递交时',
            '行动名称': '样品提交'},
           {'内容': '中标人需提交合同总价款10%的履约保证金。',
            '地点': '招标人指定地点',
            '时间': '合同签订后7个工作日内',
            '行动名称': '履约保证金提交'}],
  '内容包括': '招标项目概述、采购内容、技术和质量要求、样品提交、送达地点、交货时间、履约保证金',
  '摘要': '详细规定了北京京铁运恒采购供应站有限公司所需货物的招标要求、技术标准、质量要求、样品提交、交货细节及履约保证金等关键信息。',
  '章节标题': '第二章 招标需求',
  '章节编号': '95',
  '编制环节用途': '指导投标人根据招标要求编制投标文件，包括技术方案、质量保证、样品提交等。',
  '解读环节用途': '帮助投标人准确理解招标需求，确保投标方案符合招标人的基本质量要求和技术标准。'},
 {'关键行动': [{'内容': '投标人需在此时间前将投标文件邮寄到指定地点。',
            '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座15层',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标文件递交'},
           {'内容': '采用线上方式组织开标，投标人需按时进入会议并按照要求改名。',
            '地点': '腾讯会议线上',
            '时间': '2024年4月11日09时00分',
            '行动名称': '开标'}],
  '内容包括': '项目名称、采购数量及单位、投标报价及费用、招标代理服务费、投标保证金、投标文件组成、投标截止时间及地点、开标时间及地点、评标办法及评分标准、评标结果公示、投标保证金退还、签订合同时间、履约保证金、付款方式、投标文件有效期、招标最高限价、资金来源、质疑函接收方式、招标文件解释权。',
  '摘要': '概述了北京京铁运恒采购供应站有限公司2024年端午节物资采购项目（一级压榨花生油）的招标信息和要求。',
  '章节标题': '前附表',
  '章节编号': '125',
  '编制环节用途': '指导投标人准备和提交符合要求的投标文件，包括投标报价、保证金、文件组成等。',
  '解读环节用途': '帮助投标人理解招标的具体要求和流程，确保投标文件符合规定。'},
 {'关键行动': [{'内容': '投标人可自行踏勘现场，但需承担相关费用和责任。',
            '地点': '招标人项目现场',
            '时间': '不适用',
            '行动名称': '踏勘现场'},
           {'内容': '投标人或其他利害关系人可对招标投标活动提出质疑和投诉。',
            '地点': '有关行政监督部门',
            '时间': '知道或应当知道之日起10日内',
            '行动名称': '质疑和投诉'}],
  '内容包括': '适用范围、定义、招标方式、投标委托、踏勘现场和投标费用、联合体投标、转包、特别说明、质疑和投诉。',
  '摘要': '概述了北京京铁运恒采购供应站有限公司2024年端午节物资采购项目的招标流程和规则。',
  '章节标题': '一、总则',
  '章节编号': '130',
  '编制环节用途': '指导投标人按照招标文件要求准备和提交投标文件。',
  '解读环节用途': '帮助投标人理解招标文件内容，确保投标文件符合要求。'},
 {'关键行动': [{'内容': '投标人对招标文件有误或不合理要求提出异议',
            '地点': '书面形式',
            '时间': '递交投标文件截止时间十日前',
            '行动名称': '提出异议'},
           {'内容': '招标代理机构对招标文件进行澄清、答复、修改或补充，并发布更正公告',
            '地点': '指定信息发布媒体',
            '时间': '招标文件要求提交投标文件截止时间十五日前',
            '行动名称': '澄清、答复、修改或补充'},
           {'内容': '潜在投标人对招标文件有异议应书面提出，逾期不予受理',
            '地点': '书面形式',
            '时间': '投标截止时间十日前',
            '行动名称': '提出异议'}],
  '内容包括': '招标文件的构成、投标人风险、招标文件澄清与修改、对招标文件的异议',
  '摘要': '详细介绍了招标文件的构成、投标人风险、澄清与修改流程以及异议处理。',
  '章节标题': '二、招标文件',
  '章节编号': '159',
  '编制环节用途': '指导投标人根据招标文件要求编制投标文件，避免投标被拒绝或无效',
  '解读环节用途': '帮助投标人理解招标文件内容，确保投标文件符合要求'},
 {'关键行动': [{'内容': '投标人按照要求准备、密封和递交投标文件',
            '地点': '招标公告或投标邀请书中规定的地址',
            '时间': '投标截止期前',
            '行动名称': '提交投标文件'},
           {'内容': '投标人对投标文件进行书面修改或撤回',
            '地点': '招标人处',
            '时间': '投标截止时间前',
            '行动名称': '投标文件修改或撤回'}],
  '内容包括': '投标文件的组成要素、格式和内容要求、语言和计量单位规定、报价方式、文件有效期、保证金提交与退还规则、文件签署和装订要求、文件的密封递交及修改撤回规定、无效投标情况说明。',
  '摘要': '详细介绍了投标文件的组成、要求、语言计量、报价、有效期、保证金、签署份数、包装递交以及无效投标的情形。',
  '章节标题': '三、投标文件的编制',
  '章节编号': '175',
  '编制环节用途': '指导投标人按照规定格式和内容准备投标文件，避免因文件问题导致投标无效。',
  '解读环节用途': '帮助投标人理解投标文件的具体要求，确保投标文件的完整性和合规性。'},
 {'关键行动': [{'内容': '招标代理机构主持开标，公布投标信息，投标人法定代表人或授权代表确认投标信息',
            '地点': '规定地点',
            '时间': '规定时间',
            '行动名称': '开标会议'}],
  '内容包括': '开标准备、开标程序、投标人参与要求',
  '摘要': '规定了开标的具体流程和投标人参与开标的要求',
  '章节标题': '四、开标',
  '章节编号': '256',
  '编制环节用途': '指导投标人准备参与开标会议和确认投标信息',
  '解读环节用途': '帮助投标人理解开标流程和准备参与开标'},
 {'关键行动': [{'内容': '根据项目特点组建评标委员会，成员包括招标人代表和专家。',
            '地点': '招标单位',
            '时间': '评标前',
            '行动名称': '组建评标委员会'},
           {'内容': '评标委员会按照评标程序和原则进行评标，包括初审、实质审查与比较等。',
            '地点': '评标地点',
            '时间': '投标截止后',
            '行动名称': '评标'},
           {'内容': '评标委员会可要求投标人对投标文件中不明确或错误的内容进行澄清。',
            '地点': '评标地点',
            '时间': '评标过程中',
            '行动名称': '澄清问题'},
           {'内容': '评标委员会按照规定原则修正投标文件中的错误，并要求投标人确认。',
            '地点': '评标地点',
            '时间': '评标过程中',
            '行动名称': '错误修正'}],
  '内容包括': '评标委员会的组建、职责、评标方式、程序、澄清问题形式、错误修正原则、评标原则和办法。',
  '摘要': '介绍评标委员会的组建、职责、评标方式、程序、澄清问题形式、错误修正原则及评标原则和办法。',
  '章节标题': '五、评标',
  '章节编号': '267',
  '编制环节用途': '指导投标方准备和提交符合评标要求的投标文件。',
  '解读环节用途': '帮助投标方理解评标流程和标准，确保投标文件符合要求。'},
 {'关键行动': [{'内容': '招标代理机构将评标报告交由招标人确认',
            '地点': '招标人处',
            '时间': '评标结束后',
            '行动名称': '评标报告提交'},
           {'内容': '招标代理机构公示评标结果',
            '地点': '发布招标公告的网站',
            '时间': '评标结束后',
            '行动名称': '评标结果公示'},
           {'内容': '招标人需重新组织招标流程',
            '地点': '招标人处',
            '时间': '中标人放弃或无法履行合同时',
            '行动名称': '重新组织招标'}],
  '内容包括': '评标委员会推荐中标候选人，招标人确定中标人，公示评标结果，以及中标人放弃或无法履行合同时的处理。',
  '摘要': '本章节描述了中标候选人的推荐、中标人的确定以及中标后的相关处理流程。',
  '章节标题': '六、定标',
  '章节编号': '294',
  '编制环节用途': '指导投标方在中标后如何准备履约保证金和其他相关文件，以及在中标资格被撤销时的应对措施。',
  '解读环节用途': '帮助投标方了解中标候选人的推荐和中标人的确定流程，以及公示评标结果的重要性。'},
 {'关键行动': [{'内容': '招标人与中标人签订合同',
            '地点': '未指定',
            '时间': '《中标通知书》发出之日起30日内',
            '行动名称': '签订合同'},
           {'内容': '扣罚投标保证金并取消中标资格',
            '地点': '未指定',
            '时间': '中标人拖延或拒签合同时',
            '行动名称': '违约处罚'}],
  '内容包括': '合同签订的时间限制、中标人违约的处罚措施',
  '摘要': '规定了合同签订的时间限制和中标人违约的后果。',
  '章节标题': '七、合同授予',
  '章节编号': '298',
  '编制环节用途': '指导投标方在中标后及时签订合同，避免违约风险。',
  '解读环节用途': '明确合同签订的时间要求和违约后果，帮助投标方理解合同授予流程。'},
 {'关键行动': [{'内容': '提交合同总价款10%的履约保证金',
            '地点': '招标人指定地点',
            '时间': '签订合同后7个工作日内',
            '行动名称': '提交履约保证金'}],
  '内容包括': '中标人需在签订合同后7个工作日内提交履约保证金，金额为合同总价款的10%',
  '摘要': '规定中标人需在签订合同后7个工作日内向招标人提交合同总价款10%的履约保证金',
  '章节标题': '八、履约保证金',
  '章节编号': '300',
  '编制环节用途': '指导投标人在中标后准备和提交履约保证金',
  '解读环节用途': '确保投标人了解履约保证金的要求，以便在中标后正确履行合同'},
 {'关键行动': [],
  '内容包括': '中标人支付招标代理服务费，收费标准依据《招标代理服务收费标准》下浮12%，支付方式包括银行汇款、电汇、支票。',
  '摘要': '规定了招标代理服务费的支付责任、收费标准和支付方式。',
  '章节标题': '九、招标代理服务费的收取',
  '章节编号': '302',
  '编制环节用途': '指导投标方在中标后如何计算和支付招标代理服务费。',
  '解读环节用途': '帮助投标方了解招标代理服务费用的具体支付责任和标准，以便在投标决策中考虑相关成本。'},
 {'关键行动': [{'内容': '由招标人依法组建评标委员会，成员包括技术、经济专家和招标人代表。',
            '地点': '招标人指定地点',
            '时间': '评标前',
            '行动名称': '组建评标委员会'},
           {'内容': '评标委员会成员推举评标委员会组长，负责主持评标工作。',
            '地点': '评标地点',
            '时间': '评标开始时',
            '行动名称': '评标委员会推举组长'},
           {'内容': '评标委员会对投标文件进行资格性和符合性审查，确定通过初步评审的投标人名单。',
            '地点': '评标地点',
            '时间': '评标过程中',
            '行动名称': '初步评审'},
           {'内容': '评标委员会根据招标文件规定的评分标准，对投标人的投标文件进行评分。',
            '地点': '评标地点',
            '时间': '评标过程中',
            '行动名称': '综合评分'},
           {'内容': '评标委员会根据评分结果推荐1-3名中标候选人，由招标人确定中标人。',
            '地点': '评标地点',
            '时间': '评标结束后',
            '行动名称': '确定中标候选人'}],
  '内容包括': '评标委员会的组建、评标原则、评标内容、评标程序、评标方法、资格审查、符合性审查、评分标准。',
  '摘要': '本章节详细规定了评标委员会的组建、评标原则、评标内容、评标程序和评标方法，确保评标工作的公平、公正、科学和择优。',
  '章节标题': '第四章 评标办法及评分标准',
  '章节编号': '308',
  '编制环节用途': '指导投标方根据评标标准准备和提交投标文件，提高中标机会。',
  '解读环节用途': '帮助投标方理解评标流程和标准，确保投标文件符合要求。'},
 {'关键行动': [{'内容': '提交符合资格审查要求的相关证明材料复印件并加盖公章',
            '地点': '投标文件提交地点',
            '时间': '投标截止时间前',
            '行动名称': '提供资格审查材料'}],
  '内容包括': '资格审查的条件和证明材料要求',
  '摘要': '详细列出了投标人资格审查的具体条件和所需提供的证明材料',
  '章节标题': '附件一：资格审查表',
  '章节编号': '352',
  '编制环节用途': '指导投标人准备和提交符合要求的资格审查材料',
  '解读环节用途': '帮助投标人理解资格审查要求，确保投标文件符合规定'},
 {'关键行动': [{'内容': '根据审查表内容对投标文件进行逐项检查，确定是否符合招标文件要求。',
            '地点': '招标评审地点',
            '时间': '投标文件提交后',
            '行动名称': '符合性审查'}],
  '内容包括': '审查内容涉及授权书格式、投标保证金、投标有效期、文件密封签署盖章、资格证书和资质证明文件、对重要商务技术条款的响应、实质性响应、技术规格和性能偏离、交货期质保售后服务偏离、报价问题、附加条件、报价合理性、虚假证明材料、其他法律情形等。',
  '摘要': '详细列出了符合性审查的14项具体内容，用以评估投标文件是否符合招标文件的要求。',
  '章节标题': '附件二：符合性审查表',
  '章节编号': '355',
  '编制环节用途': '指导投标方准备和检查投标文件，确保其符合招标文件的规定，避免因不符合要求而被排除。',
  '解读环节用途': '帮助招标方明确投标文件必须满足的基本要求，确保投标文件的合规性。'},
 {'关键行动': [{'内容': '投标人需提供税务、社保等部门的证明材料，以证实其投标文件的真实性。',
            '地点': '投标文件中',
            '时间': '投标时',
            '行动名称': '提供证明材料'}],
  '内容包括': '价格分（40分）、商务分（25分）、技术分（35分）的评分细则和要求。',
  '摘要': '详细规定了投标评分的三个主要部分：价格分、商务分和技术分，以及各自的评分标准和要求。',
  '章节标题': '附件三：评分标准',
  '章节编号': '357',
  '编制环节用途': '指导投标人根据评分标准准备和编制投标文件，提高中标机会。',
  '解读环节用途': '帮助投标人理解评分标准，确保投标文件符合评分要求。'},
 {'关键行动': [{'内容': '甲乙双方根据合同条款签订买卖合同，明确双方的权利和义务。',
            '地点': '北京市丰台区北京西站东附楼瑞尔威写字楼 A1 座 A706 室',
            '时间': '中标后',
            '行动名称': '合同签订'},
           {'内容': '乙方按照合同要求将商品交付给甲方，并承担运输、装卸等费用。',
            '地点': '甲方指定地点',
            '时间': '签订合同后',
            '行动名称': '商品交付'},
           {'内容': '甲方在收到商品后10日内进行验收，乙方在质保期内负责免费更换质量问题商品。',
            '地点': '约定地点',
            '时间': '商品交付后',
            '行动名称': '验收及质保'},
           {'内容': '甲方在收到增值税专用发票后15-20日内向乙方支付60%货款，剩余40%在30日内支付。',
            '地点': '乙方银行账户',
            '时间': '商品验收合格后',
            '行动名称': '付款'}],
  '内容包括': '合同双方信息、合同条款（商品信息、质量要求、包装、交货、验收、付款、违约责任等）、合同生效条件、争议解决方式、保密条款、不可抗力、通知方式等',
  '摘要': '详细规定了买卖合同的条款和格式，包括合同的基本信息、商品信息、质量要求、包装、交货、验收、付款、违约责任等关键条款。',
  '章节标题': '第五章 合同条款及格式',
  '章节编号': '366',
  '编制环节用途': '指导投标方根据合同条款准备投标文件，包括商品报价、质量标准、包装要求、交货时间等，确保投标文件的完整性和合规性。',
  '解读环节用途': '帮助投标方理解合同的具体要求和条款，确保投标文件符合合同规定，避免违约风险。'},
 {'关键行动': [],
  '内容包括': '投标文件应包含的19个附件，涉及投标函、保证书、声明、授权书、业绩表、报价表、方案、资格声明、证明文件、法人单位信息、其他证明文件、技术规格偏离表、商务条款偏离表、人员配备表、服务费承诺书、投标保证金声明和开标会声明书等',
  '摘要': '详细列出了投标文件所需的19个附件格式和内容要求',
  '章节标题': '第六章 投标文件格式',
  '章节编号': '501',
  '编制环节用途': '指导投标方按照规定的格式和内容要求编制投标文件，提高投标文件的专业性和规范性',
  '解读环节用途': '帮助投标方理解投标文件的格式和内容要求，确保投标文件的完整性和合规性'},
 {'关键行动': [{'内容': '投标人需提交包含投标函的投标文件正本一份、副本若干及电子版。',
            '地点': '招标方指定地点',
            '时间': '投标截止日期前',
            '行动名称': '提交投标函'},
           {'内容': '投标人需支付规定的投标保证金金额，以表明其投标的严肃性。',
            '地点': '招标方指定地点',
            '时间': '投标截止日期前',
            '行动名称': '支付投标保证金'}],
  '内容包括': '投标函的提交、投标文件清单、投标保证金、合同责任、文件审查、投标有效期、撤回投标的后果、提供额外资料的承诺、通讯信息和法定代表人的签字盖章。',
  '摘要': '投标函是投标人对招标要求的正式回应，包含投标文件提交、保证金、合同责任等关键信息。',
  '章节标题': '附件 1 投标函',
  '章节编号': '515',
  '编制环节用途': '投标文件编制环节中，投标函是向招标方展示投标人资质、承诺和投标意愿的重要文件。',
  '解读环节用途': '投标函的解读帮助招标方确认投标人对招标要求的理解和接受程度，以及其投标的正式性和有效性。'},
 {'关键行动': [{'内容': '投标人需按照要求签署投标保证书，承诺遵守法律法规、保证投标内容真实性及中标后按时签订合同',
            '地点': '招标地点',
            '时间': '投标时',
            '行动名称': '签订投标保证书'}],
  '内容包括': '投标人对招标活动的法律遵守、投标内容的真实合法性、中标后合同签订的承诺及违约责任',
  '摘要': '投标人对遵守法律法规、投标内容真实性及中标后合同签订的保证',
  '章节标题': '附件 2 投标保证书',
  '章节编号': '532',
  '编制环节用途': '作为投标文件的一部分，证明投标人的法律责任和承诺，增加投标文件的正式性和可信度',
  '解读环节用途': '确保投标人理解并承诺遵守招标投标法及相关法律法规，保证投标内容的真实性和合法性，以及中标后合同签订的义务'},
 {'关键行动': [{'内容': '法定代表人或被授权代表签署声明，确认投标人在投标前三年无重大违法记录',
            '地点': '投标人所在地',
            '时间': '投标前',
            '行动名称': '签署无违法记录声明'}],
  '内容包括': '投标人需声明其在投标前三年的经营活动中没有重大违法记录，并由法定代表人或被授权代表签字或盖章确认',
  '摘要': '投标人声明其在参加项目投标活动前三年内无重大违法记录',
  '章节标题': '附件 3 无违法记录声明',
  '章节编号': '540',
  '编制环节用途': '作为投标文件的一部分，证明投标人的合规性，增加中标机会',
  '解读环节用途': '确保投标人符合无违法记录的要求，为招标人提供法律合规性保障'},
 {'关键行动': [{'内容': '法定代表人或单位负责人签署授权书，授权特定人员代表公司处理投标事务。',
            '地点': '公司注册地或项目所在地',
            '时间': '授权书生效日期',
            '行动名称': '签署授权书'},
           {'内容': '在授权书上加盖单位公章，以证明其正式性和有效性。',
            '地点': '公司注册地或项目所在地',
            '时间': '签署授权书的同时',
            '行动名称': '盖章'},
           {'内容': '将签署并盖章的授权书作为投标文件的一部分提交给招标人。',
            '地点': '招标人指定的地点',
            '时间': '投标截止日期前',
            '行动名称': '提交授权书'}],
  '内容包括': '法定代表人授权书的格式和内容，包括授权声明、授权人和被授权人的详细信息、授权书的生效日期、签字盖章等。',
  '摘要': '法定代表人授权书是一份正式文件，用于授权特定人员代表公司处理投标相关事务。',
  '章节标题': '附件 4 法定代表人授权书',
  '章节编号': '548',
  '编制环节用途': '指导投标方如何准备和提交法定代表人授权书，作为投标文件的一部分。',
  '解读环节用途': '确保投标方理解授权书的要求和格式，以便正确填写和提交。'},
 {'关键行动': [{'内容': '完成附件5表格，提供公司详细信息',
            '地点': '投标人所在地',
            '时间': '投标前',
            '行动名称': '填写投标人情况简介'}],
  '内容包括': '投标人基本信息、单位性质、财务信息、项目经验等',
  '摘要': '提供投标人的基本信息和公司详情，包括联系信息、财务状况和项目经验',
  '章节标题': '附件 5 投标人情况简介',
  '章节编号': '556',
  '编制环节用途': '让投标方展示自身实力和符合招标要求的证据',
  '解读环节用途': '帮助招标方了解投标人的资质和能力'},
 {'关键行动': [{'内容': '收集自2021年1月1日起至投标截止日同类产品销售业绩的相关证明文件',
            '地点': '投标人所在地',
            '时间': '投标截止日前',
            '行动名称': '收集销售业绩证明'},
           {'内容': '根据收集的证明文件，准确填写销售业绩表中的合同时间、项目名称、合作单位、合同金额及项目实施情况',
            '地点': '投标人所在地',
            '时间': '投标文件编制期间',
            '行动名称': '填写销售业绩表'},
           {'内容': '准备用户合同复印件、发票复印件、发票明细汇总清单、案例用户联系人及联系方式等文件',
            '地点': '投标人所在地',
            '时间': '投标文件编制期间',
            '行动名称': '准备用户合同复印件等文件'}],
  '内容包括': '合同时间、项目名称、合作单位、合同金额及项目实施情况的表格，以及用户合同复印件、发票复印件、发票明细汇总清单、案例用户联系人及联系方式',
  '摘要': '要求投标人提供自2021年1月1日起至投标截止日同类产品销售业绩的详细信息',
  '章节标题': '附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表',
  '章节编号': '559',
  '编制环节用途': '指导投标人准确填写和准备销售业绩相关的证明文件，以满足招标要求',
  '解读环节用途': '帮助投标人理解所需提供的销售业绩证明文件和信息，确保投标文件的完整性和准确性'},
 {'关键行动': [{'内容': '按照投标人须知的规定，将填写完整的投标一览表密封递交。',
            '地点': '招标人指定地点',
            '时间': '投标截止前',
            '行动名称': '密封递交投标一览表'},
           {'内容': '确保投标一览表中的投标总价与分项报价表中的总价一致。',
            '地点': '投标人所在地',
            '时间': '投标文件编制过程中',
            '行动名称': '确保投标总价一致性'},
           {'内容': '在投标报价中包含货物成本、包装费、检验检测费、仓储费、运输费、保险费、装卸费、售后服务等全部费用。',
            '地点': '投标人所在地',
            '时间': '投标文件编制过程中',
            '行动名称': '包含所有费用'}],
  '内容包括': '投标一览表的格式和内容要求，包括货物的详细信息和投标总价的书写规定。',
  '摘要': '详细列出了北京京铁运恒采购供应站有限公司2024年端午节物资采购项目（一级压榨花生油）的投标一览表，包括货物名称、规格、单价、数量、总价、交货期、交货地点、质保期等关键信息。',
  '章节标题': '附件 7 投标一览表',
  '章节编号': '562',
  '编制环节用途': '指导投标人准确填写投标一览表，包括货物的详细信息和投标总价，以及相关的签字盖章要求。',
  '解读环节用途': '帮助投标人理解投标一览表的填写要求和内容，确保投标文件的准确性和完整性。'},
 {'关键行动': [{'内容': '根据项目实际情况，填写分项报价表中的名称、规格、产地、单价、数量等信息，并计算总价。',
            '地点': '投标人所在地',
            '时间': '投标准备阶段',
            '行动名称': '填写分项报价表'},
           {'内容': '检查分项报价表中的单价和总价是否一致，确保报价的准确性。',
            '地点': '投标人所在地',
            '时间': '投标准备阶段',
            '行动名称': '核对分项报价'},
           {'内容': '投标人名称处加盖公章，法定代表人或被授权代表签字或盖章，以证明报价表的有效性。',
            '地点': '投标人所在地',
            '时间': '投标截止前',
            '行动名称': '签字盖章'}],
  '内容包括': '分项报价表的格式、填写要求和注意事项。',
  '摘要': '提供了一个详细的分项报价表模板，用于记录和计算项目的各项费用。',
  '章节标题': '附件 8 分项报价表',
  '章节编号': '570',
  '编制环节用途': '指导投标人按照规定的格式和要求填写分项报价表，明确各项费用的计算和汇总。',
  '解读环节用途': '帮助投标人理解分项报价表的填写规范和要求，确保报价的准确性和合规性。'},
 {'关键行动': [{'内容': '根据招标人需求，制定详细的投标方案',
            '地点': '投标人所在地',
            '时间': '投标前',
            '行动名称': '投标方案制定'},
           {'内容': '审核投标方案，确保其满足招标文件要求',
            '地点': '投标人所在地',
            '时间': '提交投标文件前',
            '行动名称': '投标方案审核'},
           {'内容': '提交加盖公章的投标文件',
            '地点': '招标人指定地点',
            '时间': '投标截止日期前',
            '行动名称': '投标文件提交'}],
  '内容包括': '投标方案应涵盖人员配备、货物质量保证、食品安全卫生管理、配送、售后服务、应急方案、检测报告及投标人综合实力',
  '摘要': '详细描述对招标人采购需求的理解及投标方案的制定',
  '章节标题': '附件 9 投标方案',
  '章节编号': '579',
  '编制环节用途': '指导投标人制定全面且详细的投标方案，以提高中标几率',
  '解读环节用途': '帮助投标人准确理解招标人的需求，确保投标方案的针对性和响应性'},
 {'关键行动': [{'内容': '投标人提交包含真实和有效说明、材料、文件的投标文件',
            '地点': '招标人指定地点',
            '时间': '投标截止日期',
            '行动名称': '提交投标文件'},
           {'内容': '投标人配合招标人，向银行、会计师事务所、相关企业等查证和获得有关资料',
            '地点': '相关机构和单位',
            '时间': '招标人要求时',
            '行动名称': '提供资格材料'}],
  '内容包括': '投标人响应招标邀请，承诺提供满足招标需求的货物及服务，并保证投标文件的真实性和有效性；同时表示愿意配合招标人提供进一步的资格材料。',
  '摘要': '投标人对招标邀请的响应及资格声明',
  '章节标题': '附件 10 投标人资格声明',
  '章节编号': '589',
  '编制环节用途': '指导投标人如何准备和提交资格声明，以满足招标要求。',
  '解读环节用途': '帮助招标人了解投标人的资格和承诺，确保投标文件的真实性和有效性。'},
 {'关键行动': [{'内容': '查询并打印投标人信用记录，加盖公章',
            '地点': '信用中国和中国政府采购网',
            '时间': '招标公告发布后至投标截止时间',
            '行动名称': '查询信用记录'}],
  '内容包括': '资格证明文件的具体要求和格式',
  '摘要': '列出了参与投标所需的资格证明文件清单',
  '章节标题': '附件 11 资格证明文件',
  '章节编号': '596',
  '编制环节用途': '指导投标方准备和提交符合要求的资格证明文件',
  '解读环节用途': '确保投标方理解并准备所有必要的资格证明文件'},
 {'关键行动': [{'内容': '如实填写与投标人存在单位负责人同一人或直接控股、管理关系的其他法人单位信息',
            '地点': '投标人所在地',
            '时间': '投标前',
            '行动名称': '填写附件12'},
           {'内容': '投标人名称处加盖公章，法定代表人或被授权代表签字或盖章，并填写日期',
            '地点': '投标人所在地',
            '时间': '投标前',
            '行动名称': '盖章和签字'}],
  '内容包括': '单位负责人为同一人或存在直接控股、管理关系的其他法人单位信息，包括单位名称和相互关系',
  '摘要': '要求投标人提供与其存在特定关系的其他法人单位的详细信息',
  '章节标题': '附件 12 与投标人存在“关系”的其他法人单位信息',
  '章节编号': '607',
  '编制环节用途': '指导投标人如何填写附件12，包括单位名称和相互关系，确保信息的完整性和准确性',
  '解读环节用途': '确保投标人理解并准确提供与自身存在特定关系的法人单位信息，避免因信息不全或错误导致投标无效'},
 {'关键行动': [],
  '内容包括': '附件13提供了一个平台，让投标人可以提交除了招标文件中明确要求之外的其他证明文件，以增强其投标文件的竞争力或证明其投标资格。',
  '摘要': '附件13允许投标人提交他们认为对投标有利的其他证明文件',
  '章节标题': '附件 13 投标人认为必要的其他证明文件',
  '章节编号': '611',
  '编制环节用途': '在投标文件编制环节，投标人可以利用附件13来补充和加强其投标文件，通过提供额外的证明文件来展示其优势或满足特定的招标要求。',
  '解读环节用途': '在招标文件解读环节，投标人可以识别是否有需要提交的额外证明文件，以满足或超越招标方的要求。'},
 {'关键行动': [{'内容': '根据招标规格和投标产品实际情况，如实填写技术规格偏离表，对偏差情况做出必要说明。',
            '地点': '投标人所在地',
            '时间': '投标阶段',
            '行动名称': '填写技术规格偏离表'},
           {'内容': '对填写的技术规格偏离表进行审核，确保信息准确无误，符合招标文件要求。',
            '地点': '投标人所在地',
            '时间': '投标阶段',
            '行动名称': '审核技术规格偏离表'}],
  '内容包括': '表格包含序号、产品名称、招标规格、投标规格、偏离和说明等列；要求投标人如实填写技术偏差并做出必要说明；明确了正偏离、负偏离和无偏离的标注方式。',
  '摘要': '提供了一个表格模板，用于记录和说明投标产品与招标规格之间的偏差。',
  '章节标题': '附件 14 技术规格偏离表',
  '章节编号': '612',
  '编制环节用途': '指导投标人准确填写技术规格偏离表，如实反映产品与招标规格的差异，为评标提供依据。',
  '解读环节用途': '帮助投标人理解招标规格要求，识别和评估自身产品与招标规格的偏差。'},
 {'关键行动': [{'内容': '详细填写对招标文件商务条款的正偏离、负偏离或标明无偏离',
            '地点': '投标人所在地',
            '时间': '投标准备阶段',
            '行动名称': '填写商务条款偏离表'},
           {'内容': '对填写的商务条款偏离表进行确认',
            '地点': '投标人所在地',
            '时间': '投标文件提交前',
            '行动名称': '法定代表人或被授权代表签字或盖章'}],
  '内容包括': '表格用于列出并详细说明对招标文件商务条款的正偏离、负偏离或无偏离',
  '摘要': '记录并说明投标人对招标文件商务条款的偏离情况',
  '章节标题': '附件 15 商务条款偏离表',
  '章节编号': '618',
  '编制环节用途': '允许投标人正式记录其对招标文件商务条款的任何偏离',
  '解读环节用途': '帮助投标人明确招标文件中的商务条款要求，并确定是否有偏离'},
 {'关键行动': [{'内容': '提供拟投入主要管理人和主要技术人员的社保证明复印件，并加盖公章',
            '地点': '投标人所在地',
            '时间': '近半年内任意一个月',
            '行动名称': '提供社保证明'},
           {'内容': '根据项目需求填写管理人员、技术人员和其他工作人员的数量',
            '地点': '投标人所在地',
            '时间': '投标文件准备阶段',
            '行动名称': '填写人员配备表'},
           {'内容': '投标人名称处加盖公章，法定代表人或授权代表签字或盖章',
            '地点': '投标人所在地',
            '时间': '投标文件提交前',
            '行动名称': '签字盖章'}],
  '内容包括': '项目名称、项目编号、管理人员数量、技术人员数量、其他工作人员数量、社保证明要求、投标人信息、签字盖章要求。',
  '摘要': '详细列出项目所需各类人员数量，并要求提供管理人员和技术人员的社保证明复印件。',
  '章节标题': '附件 16 拟投入本项目的人员配备表',
  '章节编号': '624',
  '编制环节用途': '指导投标方准备和填写人员配备表，附上必要的社保证明，以满足招标要求。',
  '解读环节用途': '确保投标方理解所需人员配置要求和社保证明的具体规定，以便准确响应招标文件。'},
 {'关键行动': [{'内容': '以支票、汇票或现金形式一次性支付中标服务费',
            '地点': '北京中外建工程管理有限公司',
            '时间': '签订经济合同时',
            '行动名称': '支付中标服务费'}],
  '内容包括': '承诺书内容、支付方式、支付对象、支付标准、支付时间',
  '摘要': '承诺书表明中标后将按规定支付中标服务费',
  '章节标题': '附件 17 中标服务费承诺书',
  '章节编号': '630',
  '编制环节用途': '投标方需准备承诺书以证明其支付中标服务费的意愿和能力',
  '解读环节用途': '确保投标方了解中标服务费的支付要求和标准'},
 {'关键行动': [{'内容': '提交包含账户信息的投标保证金声明，并确保信息真实有效',
            '地点': '北京中外建工程管理有限公司',
            '时间': '招投标活动结束后',
            '行动名称': '提交投标保证金声明'},
           {'内容': '及时通知招标方账户信息变更，避免保证金退还延误',
            '地点': '北京中外建工程管理有限公司',
            '时间': '账户信息变更时',
            '行动名称': '信息变更通知'}],
  '内容包括': '投标保证金退还账户信息、声明信息真实性、信息变更通知责任、后果承担、盖章要求、递交方式',
  '摘要': '声明投标保证金退还账户信息并承担信息错误导致的后果',
  '章节标题': '附件 18 关于投标保证金的声明',
  '章节编号': '636',
  '编制环节用途': '指导投标方正确填写和递交投标保证金声明',
  '解读环节用途': '确保投标方了解投标保证金退还流程和责任'},
 {'关键行动': [{'内容': '通过快递方式递交投标文件、样品及声明书',
            '地点': '指定地点',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标文件递交'},
           {'内容': '通过腾讯会议方式参与开标活动，承诺认可开标全过程',
            '地点': '腾讯会议',
            '时间': '开标时间',
            '行动名称': '参与开标活动'}],
  '内容包括': '投标文件递交方式、时间、地点；不参加现场开标活动的声明；通过腾讯会议参与开标活动的承诺；对开标全过程的认可；声明书的签署。',
  '摘要': '声明书涉及北京京铁运恒采购供应站有限公司2024年端午节物资采购项目（一级压榨花生油）的投标文件递交、开标过程认可及参与方式。',
  '章节标题': '附件 19 开标会声明书',
  '章节编号': '646',
  '编制环节用途': '指导投标人准备和递交投标文件，以及声明对开标过程的认可。',
  '解读环节用途': '确保投标人理解投标文件递交要求和开标过程，避免错过投标或不合规。'}]


In [61]:
# 大模型分析【在接下去的测试，该环节跳过，之前获得的结果直接在下面赋值给results】
#analyzer = LLMAnalyzer(prompts["chapter"], parts[19:22])
#result_test = analyzer.analyze_parts()

In [62]:
print(type(results), len(results))
from pprint import pprint
pprint(results)

<class 'list'> 37
[{'关键行动': [{'内容': '发售招标文件，售价500元人民币，需通过电汇或网上银行转账购买。',
            '地点': '北京中外建工程管理有限公司',
            '时间': '2024年3月22日至2024年3月26日，上午9:00-11:00；下午1:30-4:30',
            '行动名称': '招标文件发售'},
           {'内容': '投标人需提交20万元人民币的投标保证金。',
            '地点': '北京中外建工程管理有限公司',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标保证金提交'},
           {'内容': '投标人需在此时间前邮寄或现场递交投标文件。',
            '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座15层',
            '时间': '2024年4月11日09时00分前',
            '行动名称': '投标文件递交'},
           {'内容': '采用线上方式组织开标。',
            '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座14层',
            '时间': '2024年4月11日09时00分',
            '行动名称': '开标'}],
  '内容包括': '项目编号、采购组织类型、采购方式、招标最高限价、采购内容及技术要求、投标人资格要求、招标公告发布平台、招标文件发售细节、投标保证金要求、投标截止时间和地点、开标时间和地点、评标办法、业务咨询联系方式。',
  '摘要': '北京中外建工程管理有限公司受委托公开招标采购一级压榨花生油，明确了项目编号、采购方式、预算、技术要求、投标人资格、公告发布、招标文件发售、投标保证金、投标截止、开标时间和地点以及评标办法。',
  '章节标题': '第一章 招标公告',
  '章节编号': '46',
  '编制环节用途': '指导投标人准备和提交符合要求的投标文件，包括技术规格、商务条款和法律要求。',
  '解读环节用途': '帮助潜在投标人理解招标要

In [63]:
# 将模型分析的结果 更新 nodes
for result in results:
    print(f"章节编号: {result['章节编号']}" )
    print(f"章节标题: {result['章节标题']}")
    print(f"摘要: {result['摘要']}")
    print(f"内容包括: {result['内容包括']}")
    print(f"解读环节用途: {result['解读环节用途']}")
    print(f"编制环节用途: {result['编制环节用途']}")
    print(f"关键行动: {result['关键行动']}")
    for node in doc_structure.doc_nodes:
        if node.node_id == int(result['章节编号']):
            node.summary = result['摘要']
            node.content_include = result['内容包括']
            node.usage_in_interpretation = result['解读环节用途']
            node.usage_in_preparation = result['编制环节用途']
            node.key_actions = result['关键行动']


章节编号: 46
章节标题: 第一章 招标公告
摘要: 北京中外建工程管理有限公司受委托公开招标采购一级压榨花生油，明确了项目编号、采购方式、预算、技术要求、投标人资格、公告发布、招标文件发售、投标保证金、投标截止、开标时间和地点以及评标办法。
内容包括: 项目编号、采购组织类型、采购方式、招标最高限价、采购内容及技术要求、投标人资格要求、招标公告发布平台、招标文件发售细节、投标保证金要求、投标截止时间和地点、开标时间和地点、评标办法、业务咨询联系方式。
解读环节用途: 帮助潜在投标人理解招标要求和流程，确保投标文件符合规定。
编制环节用途: 指导投标人准备和提交符合要求的投标文件，包括技术规格、商务条款和法律要求。
关键行动: [{'内容': '发售招标文件，售价500元人民币，需通过电汇或网上银行转账购买。', '地点': '北京中外建工程管理有限公司', '时间': '2024年3月22日至2024年3月26日，上午9:00-11:00；下午1:30-4:30', '行动名称': '招标文件发售'}, {'内容': '投标人需提交20万元人民币的投标保证金。', '地点': '北京中外建工程管理有限公司', '时间': '2024年4月11日09时00分前', '行动名称': '投标保证金提交'}, {'内容': '投标人需在此时间前邮寄或现场递交投标文件。', '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座15层', '时间': '2024年4月11日09时00分前', '行动名称': '投标文件递交'}, {'内容': '采用线上方式组织开标。', '地点': '北京市丰台区福宜街5号院1号楼新时代国际中心A座14层', '时间': '2024年4月11日09时00分', '行动名称': '开标'}]
章节编号: 95
章节标题: 第二章 招标需求
摘要: 详细规定了北京京铁运恒采购供应站有限公司所需货物的招标要求、技术标准、质量要求、样品提交、交货细节及履约保证金等关键信息。
内容包括: 招标项目概述、采购内容、技术和质量要求、样品提交、送达地点、交货时间、履约保证金
解读环节用途: 帮助投标人准确理解招标需求，确保投标方案符合招标人的基本质量要求和技术标准。
编制环节用途: 指导投标人根据招标要求编制投标文件，包括技术方案、质量保证、样

In [64]:
# 检查个别节点的更新情况
for node in doc_structure.doc_nodes:
    if node.node_id == 515:
        print(node.node_id)
        print(node.element.content)
        print(node.summary)
        print(node.content_include)
        print(node.usage_in_interpretation)
        print(node.usage_in_preparation)
        print(node.key_actions)
        print(node.path_titles)

515
附件 1 投标函
投标函是投标人对招标要求的正式回应，包含投标文件提交、保证金、合同责任等关键信息。
投标函的提交、投标文件清单、投标保证金、合同责任、文件审查、投标有效期、撤回投标的后果、提供额外资料的承诺、通讯信息和法定代表人的签字盖章。
投标函的解读帮助招标方确认投标人对招标要求的理解和接受程度，以及其投标的正式性和有效性。
投标文件编制环节中，投标函是向招标方展示投标人资质、承诺和投标意愿的重要文件。
[{'内容': '投标人需提交包含投标函的投标文件正本一份、副本若干及电子版。', '地点': '招标方指定地点', '时间': '投标截止日期前', '行动名称': '提交投标函'}, {'内容': '投标人需支付规定的投标保证金金额，以表明其投标的严肃性。', '地点': '招标方指定地点', '时间': '投标截止日期前', '行动名称': '支付投标保证金'}]
['root', '[Chapter 6]:第六章 投标文件格式', '[Section 6.1]:附件 1 投标函']


<font color=orange> 文档CONTEXT构建 </font>

In [65]:
context = []
for node in doc_structure.doc_nodes:
    if node.element.is_heading:
        title_label = "Chapter" if node.element.heading_level == 1 else "Section" if node.element.heading_level == 2 else "Subsection"
        context.append(f"{title_label} {node.path_title_id} : {node.element.content}")
        context.append(f"章节路径：{'>'.join(node.path_titles[1:])}")
        if node.summary is not None:
            context.append(f"内容包括：{node.content_include}")
        if node.usage_in_interpretation is not None:
            context.append(f"解读环节用途：{node.usage_in_interpretation}")
        if node.usage_in_preparation is not None:
            context.append(f"编制环节用途：{node.usage_in_preparation}")
contexts = "\n".join(context)

In [66]:
print(type(contexts))
print(contexts)

<class 'str'>
Chapter 1 : 第一章 招标公告
章节路径：[Chapter 1]:第一章 招标公告
内容包括：项目编号、采购组织类型、采购方式、招标最高限价、采购内容及技术要求、投标人资格要求、招标公告发布平台、招标文件发售细节、投标保证金要求、投标截止时间和地点、开标时间和地点、评标办法、业务咨询联系方式。
解读环节用途：帮助潜在投标人理解招标要求和流程，确保投标文件符合规定。
编制环节用途：指导投标人准备和提交符合要求的投标文件，包括技术规格、商务条款和法律要求。
Chapter 2 : 第二章 招标需求
章节路径：[Chapter 2]:第二章 招标需求
内容包括：招标项目概述、采购内容、技术和质量要求、样品提交、送达地点、交货时间、履约保证金
解读环节用途：帮助投标人准确理解招标需求，确保投标方案符合招标人的基本质量要求和技术标准。
编制环节用途：指导投标人根据招标要求编制投标文件，包括技术方案、质量保证、样品提交等。
Chapter 3 : 第三章 投标人须知
章节路径：[Chapter 3]:第三章 投标人须知
Section 3.1 : 前附表
章节路径：[Chapter 3]:第三章 投标人须知>[Section 3.1]:前附表
内容包括：项目名称、采购数量及单位、投标报价及费用、招标代理服务费、投标保证金、投标文件组成、投标截止时间及地点、开标时间及地点、评标办法及评分标准、评标结果公示、投标保证金退还、签订合同时间、履约保证金、付款方式、投标文件有效期、招标最高限价、资金来源、质疑函接收方式、招标文件解释权。
解读环节用途：帮助投标人理解招标的具体要求和流程，确保投标文件符合规定。
编制环节用途：指导投标人准备和提交符合要求的投标文件，包括投标报价、保证金、文件组成等。
Section 3.2 : 一、总则
章节路径：[Chapter 3]:第三章 投标人须知>[Section 3.2]:一、总则
内容包括：适用范围、定义、招标方式、投标委托、踏勘现场和投标费用、联合体投标、转包、特别说明、质疑和投诉。
解读环节用途：帮助投标人理解招标文件内容，确保投标文件符合要求。
编制环节用途：指导投标人按照招标文件要求准备和提交投标文件。
Section 3.3 : 二、招标文件
章节路径：[Chapter 3]:第三章 投标人须知

In [67]:
import tiktoken
def count_tokens(text: str) -> int:
    """计算文本的token数量"""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))

In [68]:
count_tokens(contexts)

7280

<font color=orange> 思维链导入 </font>

In [69]:
# 文档路径
#doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case3_北京铁运投标人须知.docx"
COT_path = "C:/Users/huiwa/Downloads/文本分析测试/COT/COT.docx"


In [71]:
# 获取Document Elements
pipeline_COT = DocxParserPipeline(COT_path)
elements_COT = pipeline_COT.process()

2025-01-03 20:15:38,239 - docx_parser_package.docx_parser._01_xml_loader - INFO - 从 C:\Users\huiwa\Downloads\文本分析测试\COT\COT.docx 成功提取 raw XML content
2025-01-03 20:15:38,239 - docx_parser_package.docx_parser.pipeline - INFO - 成功加载文档: C:\Users\huiwa\Downloads\文本分析测试\COT\COT.docx
2025-01-03 20:15:38,245 - docx_parser_package.docx_parser._02_xml_parser - INFO - Successfully parsed XML content
2025-01-03 20:15:38,246 - docx_parser_package.docx_parser._02_xml_parser - INFO - DocxXMLParser initialized successfully
2025-01-03 20:15:38,246 - docx_parser_package.docx_parser.pipeline - INFO - 成功解析XML结构
2025-01-03 20:15:38,249 - docx_parser_package.docx_parser._03_element_extractor - INFO - Starting element extraction...
2025-01-03 20:15:38,249 - docx_parser_package.docx_parser._03_element_extractor - DEBUG - Found 24 total elements
2025-01-03 20:15:38,250 - docx_parser_package.docx_parser._03_element_extractor - DEBUG - Processing element 1, type: ElementType.PARAGRAPH
2025-01-03 20:15:38,251 - 

In [72]:
# 构建文档树
tree_builder_COT = TreeBuilder(elements_COT)
doc_structure_COT = tree_builder_COT.build_to_level(target_level=2)